In [1]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import datetime as dt

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [43]:
spacex_df['pie']= 1

In [69]:
# Create a dash application
app = dash.Dash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div([
                                    #html.H2('Select Launch Site:', style={'margin-right': '2em'}),
                                    dcc.Dropdown(id='site-dropdown',
                                                 options=[
                                                     {'label':'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label':'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label':'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label':'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                     {'label': 'ALL', 'value':'ALL'},
                                                 ],
                                                 value= 'ALL',
                                                 placeholder= 'Select a Launch Site',
                                                 searchable= True
                                                 )
                                    ]),
                                html.Br(),
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider',
                                                         min=0, max= 10000, step= 1000,
                                                         marks= {0:'0',
                                                                 2000:'2000',
                                                                 3000:'3000',
                                                                 4000:'4000',
                                                                 5000:'5000',
                                                                 6000:'6000',
                                                                 7000:'7000',
                                                                 8000:'8000',
                                                                 9000:'9000',
                                                                 1000:'1000'},
                                                         value= [min_payload, max_payload])
                                ),
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id= 'success-pie-chart', component_property= 'figure'),
    Input(component_id= 'site-dropdown', component_property= 'value')
)
def make_pie(launch_site):
    if launch_site=='ALL':  
        pie_chart= px.pie(spacex_df, values='class', names='Launch Site', title='Successful Launches for Sites')
        return pie_chart
    else:
        filtered_df= spacex_df[spacex_df['Launch Site']==launch_site]
        pie_chart= px.pie(filtered_df, values='pie', names='class', title='Successful Launches for {}'.format(launch_site))
        return pie_chart
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id= 'success-payload-scatter-chart', component_property= 'figure'),
    [Input(component_id= 'site-dropdown', component_property= 'value'), Input(component_id= 'payload-slider', component_property= 'value')]
)
def make_scatter(launch_site, payload):
    filtered_df2= spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    if launch_site=='ALL':
        scatter_plot= px.scatter(filtered_df2, x= 'Payload Mass (kg)', y='class', color= 'Booster Version Category', title= 'Correlation Between Payload and Success for all Sites')
        return scatter_plot
    else:
        filtered_df2= filtered_df2[filtered_df2['Launch Site']==launch_site]
        scatter_plot= px.scatter(filtered_df2, x= 'Payload Mass (kg)', y='class', color='Booster Version Category', title= 'Correlation Between Payload and Success for {}'.format(launch_site))
        return scatter_plot
        
# Run the app
if __name__ == '__main__':
    app.run_server()